In [ ]:

import os
import re
from datetime import datetime, date
from typing import List, Tuple, Optional
from pathlib import Path
import time

import pandas as pd
import streamlit as st
import pydeck as pdk

# Optional (only used if you click "Geocode missing coordinates")
try:
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
    GEOPY_AVAILABLE = True
except Exception:
    GEOPY_AVAILABLE = False

# --------------------
# CONFIG
# --------------------
st.set_page_config(
    page_title="Kiez Connect – Berlin Opportunities",
    page_icon="🗺️",
    layout="wide"
)

# Resolve data directory robustly (works in Streamlit & notebooks)
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "backend" / "data"

DATA_FILES = {
    "jobs": "berlin_tech_jobs.csv",
    "events": "berlin_tech_events.csv",
    "courses": "german_courses_berlin.csv",
}

BERLIN_CENTER = {"lat": 52.5200, "lon": 13.4050}

MAPBOX_TOKEN = os.getenv("MAPBOX_API_KEY", None)
MAP_STYLE = "mapbox://styles/mapbox/light-v9" if MAPBOX_TOKEN else None

# --------------------
# LOADERS & HELPERS
# --------------------
@st.cache_data(show_spinner=False)
def load_df(kind: str) -> pd.DataFrame:
    path = DATA_DIR / DATA_FILES[kind]
    if not path.exists():
        raise FileNotFoundError(f"Data file not found: {path}")

    df = pd.read_csv(path)

    # Standardize headers -> title, org, description, address, district, date, latitude, longitude, url
    col_map_candidates = [
        ("title", ["title", "job_title", "event_title", "course_title", "name"]),
        ("org", ["organization", "company", "employer", "provider", "host", "organizer"]),
        ("description", ["description", "summary", "details"]),
        ("address", ["address", "location", "venue_address"]),
        ("district", ["district", "bezirk", "kiez", "borough"]),
        ("date", ["date", "event_date", "start_date", "when"]),
        ("latitude", ["latitude", "lat"]),
        ("longitude", ["longitude", "lon", "lng", "long"]),
        ("url", ["url", "link", "website"]),
    ]
    df_cols_lower = {c.lower(): c for c in df.columns}

    for std_name, options in col_map_candidates:
        found = None
        for opt in options:
            if opt in df_cols_lower:
                found = df_cols_lower[opt]
                break
        if found is None:
            if std_name not in df.columns:
                df[std_name] = None
        else:
            if found != std_name:
                df.rename(columns={found: std_name}, inplace=True)

    # Types
    df["type"] = kind

    # Lat/Lon numeric
    for c in ["latitude", "longitude"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Parse date (if present)
    if "date" in df.columns:
        def parse_dt(x):
            if pd.isna(x) or str(x).strip() == "":
                return pd.NaT
            for fmt in ("%Y-%m-%d", "%d.%m.%Y", "%d/%m/%Y", "%Y/%m/%d", "%d-%m-%Y", "%Y-%m-%d %H:%M"):
                try:
                    return pd.to_datetime(x, format=fmt)
                except Exception:
                    continue
            try:
                return pd.to_datetime(x)
            except Exception:
                return pd.NaT
        df["date"] = df["date"].apply(parse_dt)

    return df


@st.cache_data(show_spinner=False)
def load_all() -> pd.DataFrame:
    dfs = []
    for k in DATA_FILES:
        try:
            df = load_df(k)
            dfs.append(df)
        except Exception as e:
            st.warning(f"Could not load {k}: {e}")
    if not dfs:
        return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True)


def keyword_filter(df: pd.DataFrame, query: str) -> pd.DataFrame:
    if not query:
        return df
    terms = [t.strip() for t in re.split(r"[,\s]+", query) if t.strip()]
    if not terms:
        return df
    mask = pd.Series([True] * len(df), index=df.index)
    target_cols = [c for c in ["title", "description", "org", "district", "address"] if c in df.columns]
    for term in terms:
        term_mask = pd.Series([False] * len(df), index=df.index)
        pattern = re.compile(re.escape(term), re.IGNORECASE)
        for c in target_cols:
            term_mask = term_mask | df[c].fillna("").str.contains(pattern)
        mask = mask & term_mask
    return df[mask]


def date_filter(df: pd.DataFrame, start: Optional[date], end: Optional[date]) -> pd.DataFrame:
    if start is None or end is None:
        return df
    if "date" not in df.columns or df["date"].isna().all():
        return df
    s = pd.Timestamp(start)
    e = pd.Timestamp(end) + pd.Timedelta(days=1)  # inclusive end
    return df[(df["date"] >= s) & (df["date"] < e)]


def district_filter(df: pd.DataFrame, districts: List[str]) -> pd.DataFrame:
    if not districts:
        return df
    if "district" not in df.columns:
        return df
    return df[df["district"].fillna("").str.strip().str.lower().isin([d.strip().lower() for d in districts])]


def with_coords(df: pd.DataFrame) -> pd.DataFrame:
    if "latitude" not in df.columns or "longitude" not in df.columns:
        return df.iloc[0:0]
    return df.dropna(subset=["latitude", "longitude"])


def to_pydeck_layer(df: pd.DataFrame, color: Tuple[int, int, int], get_radius=30):
    if df.empty:
        return None
    return pdk.Layer(
        "ScatterplotLayer",
        data=df,
        get_position="[longitude, latitude]",
        get_radius=get_radius,
        pickable=True,
        get_fill_color=color,
        radius_min_pixels=4,
        radius_max_pixels=16,
    )


def result_card(row: pd.Series):
    title = row.get("title") or "(untitled)"
    org = row.get("org") or ""
    desc = row.get("description") or ""
    addr = row.get("address") or ""
    dist = row.get("district") or ""
    url = row.get("url") or ""

    try:
        c = st.container(border=True)
    except TypeError:
        c = st.container()

    with c:
        st.markdown(f"**{title}**")
        sub = " · ".join([p for p in [org, dist, addr] if p])
        if sub:
            st.caption(sub)
        if isinstance(row.get("date"), pd.Timestamp):
            st.caption(f"📅 {row['date'].strftime('%Y-%m-%d')}")
        if desc:
            st.write(desc if len(desc) < 400 else desc[:400] + "…")
        if url:
            st.markdown(f"[Open link]({url})")
        cols = st.columns([1, 1, 2])
        with cols[0]:
            if st.button("⭐ Save", key=f"save_{row.name}"):
                save_favorite(row)
                st.success("Saved to Favorites")
        with cols[1]:
            if st.button("✅ Check-in", key=f"checkin_{row.name}"):
                add_checkin(row)
                st.success("Checked in! +10 points")


def save_favorite(row: pd.Series):
    favs = st.session_state.get("favorites", [])
    favs.append({
        "type": row.get("type"),
        "title": row.get("title"),
        "org": row.get("org"),
        "district": row.get("district"),
        "address": row.get("address"),
        "date": row.get("date").strftime('%Y-%m-%d') if isinstance(row.get("date"), pd.Timestamp) else "",
        "url": row.get("url"),
        "latitude": row.get("latitude"),
        "longitude": row.get("longitude"),
    })
    st.session_state["favorites"] = favs
    st.session_state["score"] = st.session_state.get("score", 0) + 5


def add_checkin(row: pd.Series):
    checkins = st.session_state.get("checkins", [])
    checkins.append({
        "type": row.get("type"),
        "title": row.get("title"),
        "when": datetime.now().strftime("%Y-%m-%d %H:%M"),
        "district": row.get("district"),
    })
    st.session_state["checkins"] = checkins
    st.session_state["score"] = st.session_state.get("score", 0) + 10


def export_passport():
    favs = st.session_state.get("favorites", [])
    chks = st.session_state.get("checkins", [])
    df_f = pd.DataFrame(favs)
    df_c = pd.DataFrame(chks)
    ts = datetime.now().strftime("%Y%m%d_%H%M")
    fav_path = BASE_DIR / f"participation_passport_favorites_{ts}.csv"
    chk_path = BASE_DIR / f"participation_passport_checkins_{ts}.csv"
    if not df_f.empty:
        df_f.to_csv(fav_path, index=False)
    if not df_c.empty:
        df_c.to_csv(chk_path, index=False)
    files = []
    if fav_path.exists():
        files.append(str(fav_path))
    if chk_path.exists():
        files.append(str(chk_path))
    return files

# --------------------
# DIAGNOSTICS
# --------------------
def diagnostics(df_all: pd.DataFrame):
    with st.expander("🔎 Diagnostics (data health)", expanded=False):
        if df_all.empty:
            st.write("No data loaded.")
            return
        counts = df_all.groupby("type").size().rename("rows").to_frame()
        coords = df_all.assign(has_coords=df_all["latitude"].notna() & df_all["longitude"].notna()) \
                       .groupby("type")["has_coords"].sum().rename("with_coords").to_frame()
        diag = counts.join(coords, how="left").fillna(0).astype(int)

        date_info = {}
        if "date" in df_all.columns and df_all["date"].notna().any():
            date_info["min_date"] = str(df_all["date"].min().date())
            date_info["max_date"] = str(df_all["date"].max().date())
        st.write("Counts & coordinates:")
        st.dataframe(diag)
        if date_info:
            st.write("Date range in data:", date_info["min_date"], "→", date_info["max_date"])
        else:
            st.write("No usable 'date' column found (or all empty).")

# --------------------
# OPTIONAL GEOCODING
# --------------------
def geocode_missing(df_in: pd.DataFrame, city_hint: str = "Berlin, Germany", limit: int = 50) -> pd.DataFrame:
    """
    Geocode up to `limit` rows that have an address but no latitude/longitude.
    Writes results back into the DataFrame and returns it. Uses Nominatim (OpenStreetMap).
    """
    if not GEOPY_AVAILABLE:
        st.error("geopy is not installed. Run: pip install geopy")
        return df_in

    df = df_in.copy()
    need = df[(df["latitude"].isna() | df["longitude"].isna()) & df["address"].notna()]
    if need.empty:
        st.success("No missing coordinates to geocode!")
        return df

    geolocator = Nominatim(user_agent="kiez_connect_app")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # be polite

    rows = need.head(limit)
    bar = st.progress(0, text="Geocoding addresses… (OpenStreetMap)")
    total = len(rows)

    for i, (idx, r) in enumerate(rows.iterrows(), start=1):
        q = str(r["address"])
        if city_hint and "berlin" not in q.lower():
            q = f"{q}, {city_hint}"
        try:
            loc = geocode(q)
            if loc:
                df.at[idx, "latitude"] = loc.latitude
                df.at[idx, "longitude"] = loc.longitude
        except Exception:
            pass
        bar.progress(i/total, text=f"Geocoding {i}/{total}…")
        time.sleep(0.1)
    bar.empty()
    st.success(f"Geocoded {min(total, limit)} addresses (saved in memory for this session).")
    return df

# --------------------
# CHATBOT (simple intent + retrieval)
# --------------------
INTENTS = {
    "jobs": ["job", "jobs", "arbeit", "stelle", "developer", "engineer", "praktikum", "internship"],
    "events": ["event", "events", "meetup", "konferenz", "conference", "hackathon", "workshop"],
    "courses": ["course", "courses", "deutsch", "german", "sprachkurs", "language", "schule", "class", "kurs"],
}

def detect_intent(user_text: str) -> str:
    t = user_text.lower()
    for intent, kws in INTENTS.items():
        if any(kw in t for kw in kws):
            return intent
    return "help"

def retrieve(user_text: str, df_all: pd.DataFrame, limit: int = 12):
    intent = detect_intent(user_text)
    if intent == "help":
        return pd.DataFrame(), intent
    subset = df_all[df_all["type"] == intent]
    stop = set(["show", "find", "near", "me", "in", "berlin", "please", "bitte", "search", "any", "some", "next", "this", "week", "month"])
    words = [w for w in re.findall(r"[A-Za-zÄÖÜäöüß0-9\-+]+", user_text) if len(w) > 3 and w.lower() not in stop]
    q = " ".join(words)
    if q:
        subset = keyword_filter(subset, q)
    if "date" in subset.columns and subset["date"].notna().any():
        subset = subset.sort_values("date", ascending=True)
    return subset.head(limit), intent

# --------------------
# UI
# --------------------
st.title("Kiez Connect – Berlin Opportunities 🗺️💬")
st.caption("Discover tech **jobs**, **events**, and **German courses** across Berlin. Save favorites, check in, and track your participation.")

# Load all data once
df_all_initial = load_all()

# Diagnostics panel to see what's loaded
diagnostics(df_all_initial)

with st.sidebar:
    st.header("Filters")

    # Type filter (default: ALL selected)
    all_types = ["jobs", "events", "courses"]
    active_types = st.multiselect("Show types", options=all_types, default=all_types)

    # Districts
    districts = sorted([d for d in df_all_initial["district"].dropna().unique()]) if "district" in df_all_initial.columns else []
    sel_districts = st.multiselect("Districts (optional)", districts)

    # Date filter defaults: use min/max from the DATA (not a fixed 2025 range)
    df_dates = df_all_initial["date"] if "date" in df_all_initial.columns else pd.Series([], dtype="datetime64[ns]")
    if df_dates.notna().any():
        min_d = df_dates.min().date()
        max_d = df_dates.max().date()
    else:
        today = date.today()
        min_d = today
        max_d = today

    use_date_filter = st.checkbox("Filter by date (events/courses)", value=df_dates.notna().any())
    start = st.date_input("Start date", value=min_d, disabled=not use_date_filter)
    end = st.date_input("End date", value=max_d, disabled=not use_date_filter)

    kw = st.text_input("Keyword filter (title/desc/org)")

    st.divider()
    st.header("Map data")
    if st.button("📍 Geocode missing coordinates", help="Adds lat/lon for rows that have addresses but no coordinates."):
        if df_all_initial.empty:
            st.info("Load data first.")
        else:
            if not GEOPY_AVAILABLE:
                st.error("geopy not installed. Run: pip install geopy")
            else:
                # Geocode within the session (lightweight cache)
                df_all_initial[:] = geocode_missing(df_all_initial)  # mutate session copy

    st.divider()
    st.header("Your Progress")
    score = st.session_state.get("score", 0)
    st.progress(min(score, 100)/100.0, text=f"Participation Score: {score} pts")

    if st.button("Export Participation Passport (CSV)"):
        files = export_passport()
        if files:
            for f in files:
                with open(f, "rb") as fh:
                    st.download_button("Download " + os.path.basename(f), data=fh.read(), file_name=os.path.basename(f))
        else:
            st.info("No favorites or check-ins yet.")

# Build filtered dataframe for map/list
df_all = df_all_initial.copy()
show_df = df_all[df_all["type"].isin(active_types)]
if kw:
    show_df = keyword_filter(show_df, kw)

if use_date_filter:
    show_df_events = show_df[show_df["type"] == "events"]
    show_df_courses = show_df[show_df["type"] == "courses"]
    show_df_other = show_df[~show_df["type"].isin(["events", "courses"])]
    show_df_events = date_filter(show_df_events, start, end)
    show_df_courses = date_filter(show_df_courses, start, end)
    show_df = pd.concat([show_df_other, show_df_events, show_df_courses], ignore_index=True)

show_df = district_filter(show_df, sel_districts)

# --------------------
# MAP
# --------------------
st.subheader("Map")
coords_df = with_coords(show_df)
jobs_df = coords_df[coords_df["type"] == "jobs"]
events_df = coords_df[coords_df["type"] == "events"]
courses_df = coords_df[coords_df["type"] == "courses"]

layers = []
ly_jobs = to_pydeck_layer(jobs_df, color=(0, 128, 255))
ly_events = to_pydeck_layer(events_df, color=(0, 200, 100))
ly_courses = to_pydeck_layer(courses_df, color=(200, 0, 150))
for ly in [ly_jobs, ly_events, ly_courses]:
    if ly is not None:
        layers.append(ly)

tooltip = {
    "html": "<b>{title}</b><br/>{org}<br/>{district}<br/>{address}",
    "style": {"backgroundColor": "white", "color": "black"}
}

if layers:
    deck = pdk.Deck(
        map_style=MAP_STYLE,
        initial_view_state=pdk.ViewState(latitude=BERLIN_CENTER["lat"], longitude=BERLIN_CENTER["lon"], zoom=10),
        layers=layers,
        tooltip=tooltip
    )
    if MAPBOX_TOKEN:
        deck.mapbox_key = MAPBOX_TOKEN
    st.pydeck_chart(deck)
else:
    st.info("No records with coordinates to show. Click **Geocode missing coordinates** (left) if your CSVs only have addresses, or add `latitude` & `longitude` columns.")

# --------------------
# CHAT
# --------------------
st.subheader("Chatbot")
if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = [
        {"role": "assistant", "content": "Hi! Tell me what you’re looking for — jobs, events, or German courses — and any keywords (e.g., 'frontend in Neukölln next month')."}
    ]

for msg in st.session_state["chat_history"]:
    with st.chat_message(msg["role"]):
        st.write(msg["content"])

user_text = st.chat_input("Type your message…")
if user_text:
    st.session_state["chat_history"].append({"role": "user", "content": user_text})
    with st.chat_message("user"):
        st.write(user_text)

    results, intent = retrieve(user_text, df_all)
    if intent == "help":
        reply = ("I can search **jobs**, **events**, or **German courses** in Berlin. "
                 "Try: *events in Kreuzberg this week* or *junior Python jobs* or *A1 German courses in Mitte*.")
        st.session_state["chat_history"].append({"role": "assistant", "content": reply})
        with st.chat_message("assistant"):
            st.write(reply)
    else:
        count = len(results)
        header = f"Found {count} {intent}."
        if count == 0:
            body = "No matches yet — try different keywords or remove some filters."
            st.session_state["chat_history"].append({"role": "assistant", "content": header + " " + body})
            with st.chat_message("assistant"):
                st.write(header)
                st.write(body)
        else:
            st.session_state["chat_history"].append({"role": "assistant", "content": header})
            with st.chat_message("assistant"):
                st.markdown(f"**{header}**")
                for _, row in results.iterrows():
                    result_card(row)

# --------------------
# LIST VIEW
# --------------------
st.subheader("Results")
if show_df.empty:
    st.write("No results with current filters.")
else:
    cols = [c for c in ["type", "title", "org", "district", "date", "address", "url"] if c in show_df.columns]
    st.dataframe(
        show_df[cols].sort_values(by=[c for c in ["type", "date", "title"] if c in cols]).reset_index(drop=True),
        use_container_width=True
    )

st.divider()
st.caption("Prototype: data-led discovery + engagement (favorites/check-ins = participation).")


2025-10-22 18:58:21.169 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 18:58:21.180 No runtime found, using MemoryCacheStorageManager
2025-10-22 18:58:21.184 No runtime found, using MemoryCacheStorageManager
2025-10-22 18:58:21.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 18:58:21.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 18:58:21.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 18:58:21.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 18:58:21.204 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 18:58:21.207 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

DeltaGenerator()

In [6]:
# app.py — chatbot + map with smarter search and on-demand geocoding

import os
import re
import time
from pathlib import Path
from typing import List, Tuple, Optional
from datetime import date

import pandas as pd
import streamlit as st
import pydeck as pdk

# Optional geocoder (used when you click "Add map pins")
try:
    from geopy.geocoders import Nominatim
    from geopy.extra.rate_limiter import RateLimiter
    GEOCODER_OK = True
except Exception:
    GEOCODER_OK = False

# -------------------- Config & paths --------------------
st.set_page_config(page_title="Kiez Connect – Berlin Opportunities", page_icon="🗺️", layout="wide")

try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "backend" / "data"

# Basenames only (we auto-detect .csv/.xlsx/.xls)
BASENAMES = {
    "jobs": "berlin_tech_jobs",
    "events": "berlin_tech_events",
    "courses": "german_courses_berlin",
}

BERLIN_CENTER = {"lat": 52.5200, "lon": 13.4050}
MAPBOX_TOKEN = os.getenv("MAPBOX_API_KEY", None)
MAP_STYLE = "mapbox://styles/mapbox/light-v9" if MAPBOX_TOKEN else None

# -------------------- File resolution & reading --------------------
def resolve_path(basename: str) -> Path:
    for ext in (".csv", ".xlsx", ".xls"):
        for pat in (basename, basename.lower(), basename.upper()):
            hits = list(DATA_DIR.glob(pat + ext))
            if hits:
                return hits[0]
    raise FileNotFoundError(f"Missing file for '{basename}' in {DATA_DIR} (csv/xlsx/xls).")

def read_any_table(path: Path) -> pd.DataFrame:
    if path.suffix.lower() == ".csv":
        try:
            return pd.read_csv(path, sep=None, engine="python", encoding="utf-8", on_bad_lines="skip")
        except Exception:
            for enc in ("utf-8-sig", "latin1"):
                try:
                    return pd.read_csv(path, sep=None, engine="python", encoding=enc, on_bad_lines="skip")
                except Exception:
                    pass
            raise
    return pd.read_excel(path)

def parse_date_any(x):
    if pd.isna(x) or str(x).strip() == "":
        return pd.NaT
    return pd.to_datetime(x, errors="coerce", dayfirst=True)

def pick(raw: pd.DataFrame, candidates: List[str], default=None):
    for name in candidates:
        if name in raw.columns:
            return raw[name]
    return pd.Series([default] * len(raw), index=raw.index)

# -------------------- Loaders (map your exact headers) --------------------
@st.cache_data(show_spinner=False)
def load_df(kind: str) -> pd.DataFrame:
    path = resolve_path(BASENAMES[kind])
    raw = read_any_table(path)
    raw = raw.rename(columns={c: str(c).strip() for c in raw.columns})  # clean headers

    df = pd.DataFrame(index=raw.index)
    df["type"] = kind
    df["description"] = None
    df["district"] = None

    # propagate any existing lat/lon if present (not in your files yet, but harmless)
    df["latitude"]  = pd.to_numeric(pick(raw, ["latitude","Latitude","lat","Lat"]), errors="coerce")
    df["longitude"] = pd.to_numeric(pick(raw, ["longitude","Longitude","lon","Lng","long"]), errors="coerce")

    if kind == "events":
        # Title | Link | Date & Time | Location
        df["title"] = pick(raw, ["Title","title","Event Title"])
        df["org"] = None
        df["address"] = pick(raw, ["Location","location"])
        df["date"] = raw["Date & Time"].apply(parse_date_any) if "Date & Time" in raw.columns else pd.NaT
        df["url"] = pick(raw, ["Link","URL","Url","link"])

    elif kind == "jobs":
        # title | company | location | job_url_direct | date_posted
        df["title"] = pick(raw, ["title","job_title","Job Title"])
        df["org"] = pick(raw, ["company","Company"])
        df["address"] = pick(raw, ["location","Location"])
        df["date"] = raw["date_posted"].apply(parse_date_any) if "date_posted" in raw.columns else pd.NaT
        df["url"] = pick(raw, ["job_url_direct","job_url","Job URL"])
        for c in ["description","job_description","summary"]:
            if c in raw.columns:
                df["description"] = raw[c]; break

    elif kind == "courses":
        # provider | course_name | location | url | (start_info optional)
        df["title"] = pick(raw, ["course_name","Course Name"])
        df["org"] = pick(raw, ["provider","Provider"])
        df["address"] = pick(raw, ["location","Location"])
        df["url"] = pick(raw, ["url","URL","Url"])
        df["date"] = raw["start_info"].apply(parse_date_any) if "start_info" in raw.columns else pd.NaT
        pieces = []
        for c in ["german_level","schedule","price","duration","phone","requirements","completion"]:
            if c in raw.columns:
                pieces.append(f"{c}: " + raw[c].astype(str))
        if pieces:
            df["description"] = pieces[0]
            for p in pieces[1:]:
                df["description"] = df["description"].fillna("") + " | " + p

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    return df

@st.cache_data(show_spinner=False)
def load_all() -> pd.DataFrame:
    return pd.concat([load_df(k) for k in BASENAMES], ignore_index=True)

# -------------------- Filtering & map helpers --------------------
def keyword_filter(df: pd.DataFrame, query: str) -> pd.DataFrame:
    if not query:
        return df
    terms = [t for t in re.split(r"[,\s]+", query) if t]
    mask = pd.Series(True, index=df.index)
    cols = [c for c in ["title","description","org","address","district"] if c in df.columns]
    for t in terms:
        patt = re.compile(re.escape(t), re.IGNORECASE)
        term_mask = pd.Series(False, index=df.index)
        for c in cols:
            term_mask = term_mask | df[c].fillna("").astype(str).str.contains(patt)
        mask = mask & term_mask
    return df[mask]

def with_coords(df: pd.DataFrame) -> pd.DataFrame:
    if "latitude" not in df or "longitude" not in df:
        return df.iloc[0:0]
    return df.dropna(subset=["latitude","longitude"])

def to_pydeck_layer(df: pd.DataFrame, color: Tuple[int,int,int]):
    if df.empty: return None
    return pdk.Layer(
        "ScatterplotLayer",
        data=df,
        get_position="[longitude, latitude]",
        get_radius=30,
        pickable=True,
        get_fill_color=color,
        radius_min_pixels=4,
        radius_max_pixels=16,
    )

# -------------------- Geocoding (addresses → lat/lon) --------------------
def geocode_missing(df_in: pd.DataFrame, limit: int = 120) -> pd.DataFrame:
    if not GEOCODER_OK:
        st.error("geopy not installed. Run: pip install geopy")
        return df_in
    df = df_in.copy()
    need = df[(df["latitude"].isna() | df["longitude"].isna()) & df["address"].notna()]
    if need.empty:
        st.success("No missing coordinates to geocode!")
        return df
    geolocator = Nominatim(user_agent="kiez_connect_app")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    rows = need.head(limit)
    bar = st.progress(0, text="Geocoding addresses…")
    total = len(rows)
    for i, (idx, r) in enumerate(rows.iterrows(), start=1):
        q = str(r["address"])
        if "berlin" not in q.lower():
            q = f"{q}, Berlin, Germany"
        try:
            loc = geocode(q)
            if loc:
                df.at[idx, "latitude"] = loc.latitude
                df.at[idx, "longitude"] = loc.longitude
        except Exception:
            pass
        bar.progress(i/total, text=f"Geocoding {i}/{total}…")
        time.sleep(0.1)
    bar.empty()
    st.success(f"Geocoded {min(total, limit)} items (stored for this session).")
    return df

# -------------------- Chatbot logic --------------------
INTENTS = {
    "jobs":    ["job","jobs","arbeit","stelle","developer","engineer","praktikum","internship"],
    "events":  ["event","events","meetup","konferenz","conference","hackathon","workshop"],
    "courses": ["course","courses","deutsch","german","sprachkurs","language","schule","class","kurs"],
}
def detect_intent(text: str) -> str:
    t = text.lower()
    for intent, kws in INTENTS.items():
        if any(k in t for k in kws):
            return intent
    return "help"

# filler words we should ignore
STOP = {
    "i","im","am","are","is","the","a","an","and","or","but","please","pls","bitte","want","know","information",
    "about","need","give","tell","show","looking","for","find","me","some","any","next","this","week","month",
}

def retrieve(text: str, df_all: pd.DataFrame, limit=12):
    intent = detect_intent(text)
    if intent == "help":
        return pd.DataFrame(), intent

    subset = df_all[df_all["type"] == intent]

    # extract keywords, but keep "berlin" (useful)
    words = [w for w in re.findall(r"[A-Za-zÄÖÜäöüß0-9\-+]+", text)
             if len(w) > 2 and w.lower() not in STOP]
    # 1) try keyword filter if we actually have non-filler words
    if words:
        filtered = keyword_filter(subset, " ".join(words))
        # 2) fallback to unfiltered if keywords killed everything
        if not filtered.empty:
            subset = filtered

    if "date" in subset and subset["date"].notna().any():
        subset = subset.sort_values("date")

    return subset.head(limit), intent

# -------------------- UI --------------------
st.title("Kiez Connect – Berlin Opportunities 🗺️💬")
st.caption("Discover tech **jobs**, **events**, and **German courses** across Berlin. Save favorites, check in, and track your participation.")

# Load data
df_loaded = load_all()

# Keep a mutable copy in session_state so geocoding can update it
if "df_all" not in st.session_state:
    st.session_state["df_all"] = df_loaded.copy()
df_all = st.session_state["df_all"]

# Diagnostics
with st.expander("🔎 Diagnostics (data health)", expanded=True):
    st.write(f"Data folder: `{DATA_DIR}`")
    counts = df_all.groupby("type").size().rename("rows").to_frame()
    if "latitude" in df_all and "longitude" in df_all:
        coords = df_all.assign(has=df_all["latitude"].notna() & df_all["longitude"].notna()) \
                       .groupby("type")["has"].sum().rename("with_coords").to_frame()
        st.dataframe(counts.join(coords, how="left").fillna(0).astype(int), use_container_width=True)
    else:
        st.dataframe(counts, use_container_width=True)
    for k in ["jobs","events","courses"]:
        st.markdown(f"**Sample – {k} (first 5 rows)**")
        st.dataframe(df_all[df_all["type"] == k].head(5), use_container_width=True)

with st.sidebar:
    st.header("Filters")
    types_all = ["jobs","events","courses"]
    active_types = st.multiselect("Show types", options=types_all, default=types_all)
    kw = st.text_input("Keyword filter (title/desc/org)")
    st.divider()
    st.header("Map pins")
    if st.button("📍 Add map pins (geocode addresses)"):
        st.session_state["df_all"] = geocode_missing(st.session_state["df_all"])
        df_all = st.session_state["df_all"]

# Build filtered set for list/map
show_df = df_all[df_all["type"].isin(active_types)]
if kw:
    show_df = keyword_filter(show_df, kw)

# Map
st.subheader("Map")
coords_df = with_coords(show_df)
layers = list(filter(None, [
    to_pydeck_layer(coords_df[coords_df["type"] == "jobs"],    (0,128,255)),
    to_pydeck_layer(coords_df[coords_df["type"] == "events"],  (0,200,100)),
    to_pydeck_layer(coords_df[coords_df["type"] == "courses"], (200,0,150)),
]))
tooltip = {"html": "<b>{title}</b><br/>{org}<br/>{address}", "style": {"backgroundColor":"white","color":"black"}}
if layers:
    deck = pdk.Deck(map_style=MAP_STYLE,
                    initial_view_state=pdk.ViewState(latitude=BERLIN_CENTER["lat"], longitude=BERLIN_CENTER["lon"], zoom=10),
                    layers=layers, tooltip=tooltip)
    if MAPBOX_TOKEN: deck.mapbox_key = MAPBOX_TOKEN
    st.pydeck_chart(deck)
else:
    st.info("No coordinates yet. Click **📍 Add map pins** to geocode addresses into lat/long.")

# Chatbot
st.subheader("Chatbot")
if "chat_history" not in st.session_state:
    st.session_state["chat_history"] = [{
        "role":"assistant",
        "content":"Hi! Say **jobs**, **events**, or **courses** + keywords (e.g., *junior Python jobs*, *events in Kreuzberg*, *A1 courses in Mitte*)."
    }]
for m in st.session_state["chat_history"]:
    with st.chat_message(m["role"]):
        st.write(m["content"])

text = st.chat_input("Type your message…")
if text:
    st.session_state["chat_history"].append({"role":"user","content":text})
    with st.chat_message("user"): st.write(text)
    results, intent = retrieve(text, df_all)
    with st.chat_message("assistant"):
        if intent == "help":
            st.write("I can search **jobs**, **events**, or **German courses** in Berlin. Try: *junior Python jobs* or *events in Kreuzberg* or *A1 courses in Mitte*.")
        else:
            st.markdown(f"**Found {len(results)} {intent}.**")
            if results.empty:
                st.write("No matches yet — try different keywords or clear the filter at left.")
            else:
                for _, r in results.iterrows():
                    title = r.get("title") or "(untitled)"
                    org = r.get("org") or ""
                    addr = r.get("address") or ""
                    url = r.get("url") or ""
                    line = f"- **{title}**"
                    if org:  line += f" · {org}"
                    if addr: line += f" · {addr}"
                    if pd.notna(r.get("date")): line += f" · {pd.to_datetime(r['date']).strftime('%Y-%m-%d')}"
                    st.markdown(line)
                    if url: st.markdown(f"[Open link]({url})")

# Results table
st.subheader("Results")
if show_df.empty:
    st.write("No results with current filters.")
else:
    cols = [c for c in ["type","title","org","address","date","url"] if c in show_df.columns]
    st.dataframe(show_df[cols].sort_values(by=[c for c in ["type","date","title"] if c in cols]).reset_index(drop=True),
                 use_container_width=True)

st.divider()
st.caption("Smarter search + on-demand geocoding. Pins appear after geocoding; chat works even without pins.")


2025-10-22 19:26:30.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 19:26:30.335 No runtime found, using MemoryCacheStorageManager
2025-10-22 19:26:30.345 No runtime found, using MemoryCacheStorageManager
2025-10-22 19:26:30.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 19:26:30.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 19:26:30.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 19:26:30.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 19:26:30.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 19:26:30.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

DeltaGenerator()

In [12]:
!pip install streamlit streamlit-folium folium pandas geopy openpyxl

Defaulting to user installation because normal site-packages is not writeable
  Using cached streamlit_folium-0.25.3-py3-none-any.whl.metadata (2.1 kB)
  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached branca-0.8.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached xyzservices-2025.4.0-py3-none-any.whl.metadata (4.3 kB)
Using cached streamlit_folium-0.25.3-py3-none-any.whl (524 kB)
Using cached folium-0.20.0-py2.py3-none-any.whl (113 kB)
Using cached branca-0.8.2-py3-none-any.whl (26 kB)
Using cached xyzservices-2025.4.0-py3-none-any.whl (90 kB)

   ---------------------------------------- 0/4 [xyzservices]
   ---------- ----------------------------- 1/4 [branca]
   -------------------- ------------------- 2/4 [folium]
   -------------------- ------------------- 2/4 [folium]
   -------------------- ------------------- 2/4 [folium]
   -------------------- ------------------- 2/4 [folium]
   -------------------- ------------------- 2/4 [folium]
   --------

In [23]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium

# ------------------------------
# Load Data (pre-geocoded)
# ------------------------------
@st.cache_data
def load_data():
    base = "backend/data/"
    jobs = pd.read_csv(base + "berlin_tech_jobs.csv")
    events = pd.read_csv(base + "berlin_tech_events.csv")
    courses = pd.read_csv(base + "german_courses_berlin.csv")

    for df, t in [(jobs, "job"), (events, "event"), (courses, "course")]:
        df.columns = [c.strip().lower() for c in df.columns]
        df["type"] = t

    return pd.concat([jobs, events, courses], ignore_index=True)

df = load_data()

# ------------------------------
# Streamlit Page Setup
# ------------------------------
st.set_page_config(page_title="Kiez Connect", page_icon="💬", layout="wide")

st.title("💬 Kiez Connect – Your Berlin Chat Assistant")
st.caption("Ask about **jobs**, **events**, or **German courses** anywhere in Berlin — get instant results.")

col_chat, col_map = st.columns([0.7, 0.3])

# ------------------------------
# Initialize session safely
# ------------------------------
if "results" not in st.session_state:
    st.session_state.results = df.copy()

# ------------------------------
# CHAT SECTION
# ------------------------------
with col_chat:
    user_msg = st.chat_input("Example: 'show me jobs in Berlin Mitte' or 'find events in Kreuzberg'")

    if user_msg:
        st.chat_message("user").write(user_msg)
        msg = user_msg.lower()

        # Detect topic automatically
        topic = None
        if "job" in msg or "work" in msg:
            topic = "job"
        elif "event" in msg or "meetup" in msg:
            topic = "event"
        elif "course" in msg or "german" in msg or "class" in msg:
            topic = "course"

        # Detect Berlin district
        districts = [
            "mitte", "kreuzberg", "neukölln", "friedrichshain", "charlottenburg",
            "schöneberg", "tempelhof", "pankow", "spandau", "steglitz", "treptow",
            "köpenick", "marzahn", "hellersdorf", "reinickendorf", "prenzlauer berg"
        ]
        location = next((d for d in districts if d in msg), None)

        # Detect keyword (optional)
        common_keywords = ["developer", "engineer", "data", "design", "marketing", "teacher", "python", "manager"]
        keyword = next((k for k in common_keywords if k in msg), None)

        # --- Filter Data ---
        subset = df.copy()
        if topic:
            subset = subset[subset["type"] == topic]
        if location:
            subset = subset[subset["location"].fillna("").str.contains(location, case=False)]
        if keyword:
            subset = subset[subset["title"].fillna("").str.contains(keyword, case=False)]

        st.session_state.results = subset

        # --- Respond instantly ---
        if subset.empty:
            st.chat_message("assistant").write("❌ Sorry, I couldn't find any results for that request. Try another location or keyword.")
        else:
            if topic == "job":
                emoji = "💼"
            elif topic == "event":
                emoji = "🎉"
            elif topic == "course":
                emoji = "🎓"
            else:
                emoji = "📍"

            area = location.title() if location else "Berlin"
            st.chat_message("assistant").write(f"{emoji} Found **{len(subset)} {topic}s** around **{area}**:")

            # Show clickable results
            for _, row in subset.head(8).iterrows():
                title = row.get("title", "No title")
                loc = row.get("location", "")
                # --- Add clickable links depending on type ---
                link = ""
                if topic == "job":
                    link = row.get("job_url_direct") or row.get("job_url")
                elif topic == "event":
                    link = row.get("link")
                elif topic == "course":
                    link = row.get("url") or row.get("booking_url")

                if pd.notna(link) and isinstance(link, str) and link.startswith("http"):
                    st.markdown(f"• **[{title}]({link})** — {loc}")
                else:
                    st.markdown(f"• **{title}** — {loc}")

# ------------------------------
# MAP SECTION (sidebar column)
# ------------------------------
with col_map:
    st.subheader("🗺️ Map (Optional)")
    if st.toggle("Show map of results"):
        results = st.session_state.results
        m = folium.Map(location=[52.5200, 13.4050], zoom_start=11)
        color_map = {"job": "blue", "event": "green", "course": "orange"}
        points = results.dropna(subset=["latitude", "longitude"])

        if not points.empty:
            for _, r in points.iterrows():
                color = color_map.get(r["type"], "gray")
                title = str(r.get("title", "No title"))
                loc = str(r.get("location", "Berlin"))
                folium.CircleMarker(
                    [r["latitude"], r["longitude"]],
                    radius=6,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.9,
                    popup=f"<b>{title}</b><br>{loc}",
                    tooltip=f"{r['type'].title()}: {title}"
                ).add_to(m)
            sw = [points["latitude"].min(), points["longitude"].min()]
            ne = [points["latitude"].max(), points["longitude"].max()]
            m.fit_bounds([sw, ne])
        else:
            folium.Marker([52.5200, 13.4050], popup="Berlin Center").add_to(m)
        st_folium(m, height=400, width=None)

# ------------------------------
# RESET CHAT BUTTON
# ------------------------------
st.divider()
if st.button("🔄 Clear Chat"):
    st.session_state.results = df.copy()
    st.rerun()


2025-10-22 21:12:17.930 No runtime found, using MemoryCacheStorageManager
2025-10-22 21:12:17.940 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.943 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.954 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:12:17.960 Thread 'MainThread':

In [25]:
import pandas as pd
df = pd.read_csv("backend/data/berlin_tech_jobs.csv")
print(df.columns)
print(df["location"].head())
print(df["title"].head())

Index(['id', 'site', 'job_url', 'job_url_direct', 'title', 'company',
       'location', 'date_posted', 'job_type', 'is_remote', 'company_industry',
       'company_url_direct', 'district', 'latitude', 'longitude'],
      dtype='object')
0    Berlin, BE, DE
1    Berlin, BE, DE
2    Berlin, BE, DE
3    Berlin, BE, DE
4    Berlin, BE, DE
Name: location, dtype: object
0                      Data & Business Systems Analyst
1                         Working Student Data Analyst
2         CRM System Specialist & Data Analyst (w/m/d)
3      Senior Data Scientist (Consumer, Global Search)
4    Werkstudent (w/m/d) Data Analyst in der Grobpl...
Name: title, dtype: object


In [30]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium
from urllib.parse import quote_plus

# ------------------------------
# Load Data (pre-geocoded)
# ------------------------------
@st.cache_data
def load_data():
    base = "backend/data/"
    jobs = pd.read_csv(base + "berlin_tech_jobs.csv")
    events = pd.read_csv(base + "berlin_tech_events.csv")
    courses = pd.read_csv(base + "german_courses_berlin.csv")

    for df, t in [(jobs, "job"), (events, "event"), (courses, "course")]:
        df.columns = [c.strip().lower() for c in df.columns]
        df["type"] = t

    return pd.concat([jobs, events, courses], ignore_index=True)

df = load_data()

# ------------------------------
# Streamlit Page Setup
# ------------------------------
st.set_page_config(page_title="Kiez Connect", page_icon="💬", layout="wide")

st.title("💬 Kiez Connect – Your Berlin Chat Assistant")
st.caption("Ask about **jobs**, **events**, or **German courses** anywhere in Berlin — get instant results.")

col_chat, col_map = st.columns([0.7, 0.3])

# ------------------------------
# Initialize session safely
# ------------------------------
if "results" not in st.session_state:
    st.session_state.results = df.copy()

# ------------------------------
# CHAT SECTION
# ------------------------------
with col_chat:
    user_msg = st.chat_input("Example: 'show me jobs in Berlin Mitte' or 'find events in Kreuzberg'")

    if user_msg:
        st.chat_message("user").write(user_msg)
        msg = user_msg.lower()

        # Detect topic automatically
        topic = None
        if "job" in msg or "work" in msg:
            topic = "job"
        elif "event" in msg or "meetup" in msg:
            topic = "event"
        elif "course" in msg or "german" in msg or "class" in msg:
            topic = "course"

        # Detect Berlin district
        districts = [
            "mitte", "kreuzberg", "neukölln", "friedrichshain", "charlottenburg",
            "schöneberg", "tempelhof", "pankow", "spandau", "steglitz",
            "treptow", "köpenick", "marzahn", "hellersdorf", "reinickendorf", "prenzlauer berg"
        ]
        location = next((d for d in districts if d in msg), None)

        # Detect keyword (optional)
        common_keywords = ["developer", "engineer", "data", "design", "marketing", "teacher", "python", "manager"]
        keyword = next((k for k in common_keywords if k in msg), None)

        # --- Filter Data (Improved: use district + location + fallback) ---
        subset = df.copy()

        # 1️⃣ Filter by topic
        if topic:
            subset = subset[subset["type"] == topic]

        # 2️⃣ Filter by location or district
        if location:
            loc = location.lower()
            subset = subset[
                subset["district"].fillna("").str.lower().str.contains(loc)
                | subset["location"].fillna("").str.lower().str.contains(loc)
                | subset["location"].fillna("").str.lower().str.contains("berlin")
            ]

        # 3️⃣ Filter by keyword (title / company / provider)
        if keyword:
            key = keyword.lower()
            search_cols = [c for c in ["title", "company", "provider", "course_name"] if c in subset.columns]
            subset = subset[
                subset[search_cols].apply(
                    lambda x: x.astype(str).str.lower().str.contains(key).any(), axis=1
                )
            ]

        st.session_state.results = subset

        # --- Respond instantly ---
        if subset.empty:
            st.chat_message("assistant").write("❌ Sorry, I couldn't find any results for that request. Try another location or keyword.")
        else:
            if topic == "job":
                emoji = "💼"
            elif topic == "event":
                emoji = "🎉"
            elif topic == "course":
                emoji = "🎓"
            else:
                emoji = "📍"

            area = location.title() if location else "Berlin"
            st.chat_message("assistant").write(f"{emoji} Found **{len(subset)} {topic}s** around **{area}**:")

            # Show clickable results
            for _, row in subset.head(8).iterrows():
                title = row.get("title", "No title")
                loc = row.get("district") or row.get("location", "")
                # --- Add clickable links depending on type ---
                link = ""
                if topic == "job":
                    link = row.get("job_url_direct") or row.get("job_url")
                elif topic == "event":
                    link = row.get("link")
                elif topic == "course":
                    link = row.get("url") or row.get("booking_url")

                if pd.notna(link) and isinstance(link, str) and link.startswith("http"):
                    st.markdown(f"• **[{title}]({link})** — {loc}")
                else:
                    st.markdown(f"• **{title}** — {loc}")

# ------------------------------
# MAP SECTION (sidebar column)
# ------------------------------
with col_map:
    st.subheader("🗺️ Map (Optional)")
    if st.toggle("Show map of results"):
        results = st.session_state.results
        m = folium.Map(location=[52.5200, 13.4050], zoom_start=11)
        color_map = {"job": "blue", "event": "green", "course": "orange"}
        points = results.dropna(subset=["latitude", "longitude"])

        if not points.empty:
            for _, r in points.iterrows():
                color = color_map.get(r["type"], "gray")
                title = str(r.get("title", "No title"))
                loc = str(r.get("district") or r.get("location", "Berlin"))
                folium.CircleMarker(
                    [r["latitude"], r["longitude"]],
                    radius=6,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.9,
                    popup=f"<b>{title}</b><br>{loc}",
                    tooltip=f"{r['type'].title()}: {title}"
                ).add_to(m)
            sw = [points["latitude"].min(), points["longitude"].min()]
            ne = [points["latitude"].max(), points["longitude"].max()]
            m.fit_bounds([sw, ne])
        else:
            folium.Marker([52.5200, 13.4050], popup="Berlin Center").add_to(m)

        st_folium(m, height=400, width=None)

        # ------------------------------
        # GOOGLE MAPS BUTTON
        # ------------------------------
        if not results.empty:
            if {"latitude", "longitude"}.issubset(results.columns) and results["latitude"].notna().any():
                coords = results.dropna(subset=["latitude", "longitude"]).head(15)
                maps_query = "|".join(
                    [f"{r['latitude']},{r['longitude']}" for _, r in coords.iterrows()]
                )
            else:
                # Fallback: use location names
                maps_query = "|".join(
                    [quote_plus(str(r["district"] or r["location"])) for _, r in results.head(15).iterrows()]
                )
            gmaps_url = f"https://www.google.com/maps/search/?api=1&query={maps_query}"
            st.markdown(f"[📍 Open these locations in Google Maps]({gmaps_url})", unsafe_allow_html=True)

# ------------------------------
# RESET CHAT BUTTON
# ------------------------------
st.divider()
if st.button("🔄 Clear Chat"):
    st.session_state.results = df.copy()
    st.rerun()


2025-10-22 21:32:35.360 No runtime found, using MemoryCacheStorageManager
2025-10-22 21:32:35.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.375 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.383 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:32:35.384 Thread 'MainThread':

In [32]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium
from urllib.parse import quote_plus
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from functools import lru_cache

# ------------------------------
# Live geocoder (cached)
# ------------------------------
@lru_cache(maxsize=200)
def quick_geocode(address: str):
    """Return (lat, lon) for a given address using Nominatim with caching."""
    if not address:
        return None, None
    try:
        geolocator = Nominatim(user_agent="kiez_connect_livegeo")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        location = geocode(f"{address}, Berlin, Germany")
        if location:
            return location.latitude, location.longitude
    except Exception:
        pass
    return None, None

# ------------------------------
# Load Data
# ------------------------------
@st.cache_data
def load_data():
    base = "backend/data/"
    jobs = pd.read_csv(base + "berlin_tech_jobs.csv")
    events = pd.read_csv(base + "berlin_tech_events.csv")
    courses = pd.read_csv(base + "german_courses_berlin.csv")

    for df, t in [(jobs, "job"), (events, "event"), (courses, "course")]:
        df.columns = [c.strip().lower() for c in df.columns]
        df["type"] = t

    return pd.concat([jobs, events, courses], ignore_index=True)

df = load_data()

# ------------------------------
# Streamlit Page Setup
# ------------------------------
st.set_page_config(page_title="Kiez Connect", page_icon="💬", layout="wide")

st.title("💬 Kiez Connect – Your Berlin Chat Assistant")
st.caption("Ask about **jobs**, **events**, or **German courses** anywhere in Berlin — get instant results.")

col_chat, col_map = st.columns([0.7, 0.3])

# ------------------------------
# Initialize session safely
# ------------------------------
if "results" not in st.session_state:
    st.session_state.results = df.copy()

# ------------------------------
# CHAT SECTION
# ------------------------------
with col_chat:
    user_msg = st.chat_input("Example: 'show me jobs in Berlin Mitte' or 'find events in Kreuzberg'")

    if user_msg:
        st.chat_message("user").write(user_msg)
        msg = user_msg.lower()

        # Detect topic automatically
        topic = None
        if "job" in msg or "work" in msg:
            topic = "job"
        elif "event" in msg or "meetup" in msg:
            topic = "event"
        elif "course" in msg or "german" in msg or "class" in msg:
            topic = "course"

        # Detect Berlin district
        districts = [
            "mitte", "kreuzberg", "neukölln", "friedrichshain", "charlottenburg",
            "schöneberg", "tempelhof", "pankow", "spandau", "steglitz",
            "treptow", "köpenick", "marzahn", "hellersdorf", "reinickendorf", "prenzlauer berg"
        ]
        location = next((d for d in districts if d in msg), None)

        # Detect keyword (optional)
        common_keywords = ["developer", "engineer", "data", "design", "marketing", "teacher", "python", "manager"]
        keyword = next((k for k in common_keywords if k in msg), None)

        # --- Filter Data (Improved: use district + location + fallback) ---
        subset = df.copy()

        # 1️⃣ Filter by topic
        if topic:
            subset = subset[subset["type"] == topic]

        # 2️⃣ Filter by location or district
        if location:
            loc = location.lower()
            subset = subset[
                subset["district"].fillna("").str.lower().str.contains(loc)
                | subset["location"].fillna("").str.lower().str.contains(loc)
                | subset["location"].fillna("").str.lower().str.contains("berlin")
            ]

        # 3️⃣ Filter by keyword (title / company / provider)
        if keyword:
            key = keyword.lower()
            search_cols = [c for c in ["title", "company", "provider", "course_name"] if c in subset.columns]
            subset = subset[
                subset[search_cols].apply(
                    lambda x: x.astype(str).str.lower().str.contains(key).any(), axis=1
                )
            ]

        st.session_state.results = subset

        # --- Respond instantly ---
        if subset.empty:
            st.chat_message("assistant").write("❌ Sorry, I couldn't find any results for that request. Try another location or keyword.")
        else:
            if topic == "job":
                emoji = "💼"
            elif topic == "event":
                emoji = "🎉"
            elif topic == "course":
                emoji = "🎓"
            else:
                emoji = "📍"

            area = location.title() if location else "Berlin"
            st.chat_message("assistant").write(f"{emoji} Found **{len(subset)} {topic}s** around **{area}**:")

            # Show clickable results
            for _, row in subset.head(8).iterrows():
                title = row.get("title", "No title")
                loc = row.get("district") or row.get("location", "")
                # --- Add clickable links depending on type ---
                link = ""
                if topic == "job":
                    link = row.get("job_url_direct") or row.get("job_url")
                elif topic == "event":
                    link = row.get("link")
                elif topic == "course":
                    link = row.get("url") or row.get("booking_url")

                if pd.notna(link) and isinstance(link, str) and link.startswith("http"):
                    st.markdown(f"• **[{title}]({link})** — {loc}")
                else:
                    st.markdown(f"• **{title}** — {loc}")

# ------------------------------
# MAP SECTION (sidebar column)
# ------------------------------
with col_map:
    st.subheader("🗺️ Map (Optional)")
    if st.toggle("Show map of results"):
        results = st.session_state.results.copy()

        # --- Fill missing coordinates on the fly ---
        if {"latitude", "longitude"}.issubset(results.columns):
            for i, row in results.iterrows():
                if pd.isna(row.get("latitude")) or pd.isna(row.get("longitude")):
                    addr = row.get("district") or row.get("location", "")
                    lat, lon = quick_geocode(addr)
                    results.at[i, "latitude"] = lat
                    results.at[i, "longitude"] = lon
        else:
            results["latitude"], results["longitude"] = zip(*[
                quick_geocode(row.get("district") or row.get("location", ""))
                for _, row in results.iterrows()
            ])

        # --- Build map ---
        m = folium.Map(location=[52.5200, 13.4050], zoom_start=11)
        color_map = {"job": "blue", "event": "green", "course": "orange"}
        points = results.dropna(subset=["latitude", "longitude"])

        if not points.empty:
            for _, r in points.iterrows():
                color = color_map.get(r["type"], "gray")
                title = str(r.get("title", "No title"))
                loc = str(r.get("district") or r.get("location", "Berlin"))
                folium.CircleMarker(
                    [r["latitude"], r["longitude"]],
                    radius=6,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.9,
                    popup=f"<b>{title}</b><br>{loc}",
                    tooltip=f"{r['type'].title()}: {title}"
                ).add_to(m)
            sw = [points["latitude"].min(), points["longitude"].min()]
            ne = [points["latitude"].max(), points["longitude"].max()]
            m.fit_bounds([sw, ne])
        else:
            folium.Marker([52.5200, 13.4050], popup="Berlin Center").add_to(m)

        st_folium(m, height=400, width=None)

        # --- Google Maps Button ---
        if not results.empty:
            if {"latitude", "longitude"}.issubset(results.columns) and results["latitude"].notna().any():
                coords = results.dropna(subset=["latitude", "longitude"]).head(15)
                maps_query = "|".join(
                    [f"{r['latitude']},{r['longitude']}" for _, r in coords.iterrows()]
                )
            else:
                maps_query = "|".join(
                    [quote_plus(str(r["district"] or r["location"])) for _, r in results.head(15).iterrows()]
                )
            gmaps_url = f"https://www.google.com/maps/search/?api=1&query={maps_query}"
            st.markdown(f"[📍 Open these locations in Google Maps]({gmaps_url})", unsafe_allow_html=True)

# ------------------------------
# RESET CHAT BUTTON
# ------------------------------
st.divider()
if st.button("🔄 Clear Chat"):
    st.session_state.results = df.copy()
    st.rerun()


2025-10-22 21:43:20.335 No runtime found, using MemoryCacheStorageManager
2025-10-22 21:43:20.345 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.354 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 21:43:20.366 Thread 'MainThread':

In [ ]:
import streamlit as st
import pandas as pd
import pydeck as pdk
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from functools import lru_cache
from urllib.parse import quote_plus

# ------------------------------
# Quick live geocoder (cached)
# ------------------------------
@lru_cache(maxsize=300)
def quick_geocode(address: str):
    if not address:
        return None, None
    try:
        geolocator = Nominatim(user_agent="kiez_connect_livegeo")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        loc = geocode(f"{address}, Berlin, Germany")
        if loc:
            return loc.latitude, loc.longitude
    except Exception:
        pass
    return None, None


# ------------------------------
# Load Data
# ------------------------------
@st.cache_data
def load_data():
    base = "backend/data/"
    jobs = pd.read_csv(base + "berlin_tech_jobs.csv")
    events = pd.read_csv(base + "berlin_tech_events.csv")
    courses = pd.read_csv(base + "german_courses_berlin.csv")

    for df, t in [(jobs, "job"), (events, "event"), (courses, "course")]:
        df.columns = [c.strip().lower() for c in df.columns]
        df["type"] = t

    return pd.concat([jobs, events, courses], ignore_index=True)


df = load_data()

# ------------------------------
# Streamlit Page setup
# ------------------------------
st.set_page_config(page_title="Kiez Connect", page_icon="💬", layout="wide")

st.title("💬 Kiez Connect – Your Berlin Chat Assistant")
st.caption("Ask about **jobs**, **events**, or **German courses** in Berlin. Get instant results and map pins.")

# Initialize state
if "results" not in st.session_state:
    st.session_state.results = df.copy()

# Layout: two columns (list + map)
col_list, col_map = st.columns([0.45, 0.55])

# ------------------------------
# Chat input
# ------------------------------
query = st.chat_input("Try: 'jobs in Mitte', 'events in Kreuzberg', or 'German courses in Charlottenburg'")

if query:
    st.chat_message("user").write(query)
    q = query.lower()

    # --- detect topic ---
    topic = None
    if "job" in q or "work" in q:
        topic = "job"
    elif "event" in q or "meetup" in q or "conference" in q:
        topic = "event"
    elif "course" in q or "german" in q or "class" in q:
        topic = "course"

    # --- detect location ---
    districts = [
        "mitte", "kreuzberg", "neukölln", "friedrichshain", "charlottenburg",
        "schöneberg", "tempelhof", "pankow", "spandau", "steglitz",
        "treptow", "köpenick", "marzahn", "hellersdorf", "reinickendorf",
        "prenzlauer berg", "wilmersdorf", "moabit", "wedding"
    ]
    location = next((d for d in districts if d in q), None)

    # --- detect keyword ---
    keywords = ["developer", "engineer", "data", "design", "marketing", "teacher", "python", "manager", "tech", "startup"]
    keyword = next((k for k in keywords if k in q), None)

    # --- Filter Data (district + location + fallback Berlin + keyword)
    subset = df.copy()
    if topic:
        subset = subset[subset["type"] == topic]

    if location:
        loc = location.lower()
        subset = subset[
            subset["district"].fillna("").str.lower().str.contains(loc)
            | subset["location"].fillna("").str.lower().str.contains(loc)
            | subset["location"].fillna("").str.lower().str.contains("berlin")
        ]
    else:
        subset = subset[
            subset["location"].fillna("").str.lower().str.contains("berlin")
            | subset["district"].notna()
        ]

    if keyword:
        key = keyword.lower()
        search_cols = [c for c in ["title", "company", "provider", "course_name"] if c in subset.columns]
        if search_cols:
            subset = subset[
                subset[search_cols].apply(lambda x: x.astype(str).str.lower().str.contains(key).any(), axis=1)
            ]

    st.session_state.results = subset

    if subset.empty:
        st.chat_message("assistant").write("❌ No matches found. Try another Berlin district or keyword.")
    else:
        emoji = {"job": "💼", "event": "🎉", "course": "🎓"}.get(topic, "📍")
        area = location.title() if location else "Berlin"
        st.chat_message("assistant").write(
            f"{emoji} Found **{len(subset)} {topic+'s' if topic else 'results'}** around **{area}**."
        )

# ------------------------------
# LEFT COLUMN: Results list
# ------------------------------
with col_list:
    results = st.session_state.results.copy()
    st.subheader("Results")

    if results.empty:
        st.info("No results yet. Ask about jobs, events, or German courses in Berlin.")
    else:
        for idx, row in results.head(20).iterrows():
            title = row.get("title", "No title")
            loc = row.get("district") or row.get("location", "Berlin")
            link = ""

            if row["type"] == "job":
                link = row.get("job_url_direct") or row.get("job_url")
            elif row["type"] == "event":
                link = row.get("link")
            elif row["type"] == "course":
                link = row.get("url") or row.get("booking_url")

            if pd.notna(link) and isinstance(link, str) and link.startswith("http"):
                st.markdown(f"**[{title}]({link})**  \n{loc}")
            else:
                st.markdown(f"**{title}**  \n{loc}")
            st.markdown("---")

# ------------------------------
# RIGHT COLUMN: Map (PyDeck)
# ------------------------------
with col_map:
    st.subheader("📍 Map View")

    results = st.session_state.results.copy()

    if not results.empty:
        # Fill missing coordinates dynamically
        if {"latitude", "longitude"}.issubset(results.columns):
            for i, row in results.iterrows():
                if pd.isna(row.get("latitude")) or pd.isna(row.get("longitude")):
                    addr = row.get("district") or row.get("location", "")
                    lat, lon = quick_geocode(addr)
                    results.at[i, "latitude"] = lat
                    results.at[i, "longitude"] = lon
        else:
            results["latitude"], results["longitude"] = zip(*[
                quick_geocode(row.get("district") or row.get("location", ""))
                for _, row in results.iterrows()
            ])

        points = results.dropna(subset=["latitude", "longitude"]).copy()

        if not points.empty:
            # Color by type
            def color_for_type(t):
                t = (t or "").lower()
                return {
                    "job": [0, 122, 255],
                    "event": [0, 170, 90],
                    "course": [255, 140, 0],
                }.get(t, [120, 120, 120])

            points["color"] = points["type"].apply(color_for_type)

            mid_lat = float(points["latitude"].mean())
            mid_lon = float(points["longitude"].mean())

            layer = pdk.Layer(
                "ScatterplotLayer",
                data=points,
                get_position="[longitude, latitude]",
                get_fill_color="color",
                get_radius=80,
                pickable=True,
                opacity=0.85,
            )

            tooltip = {
                "html": "<b>{title}</b><br/>{district} {location}",
                "style": {"backgroundColor": "white", "color": "black"},
            }

            view_state = pdk.ViewState(latitude=mid_lat, longitude=mid_lon, zoom=11)
            deck = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip, map_style="light")

            st.pydeck_chart(deck, use_container_width=True)

            # --- Legend ---
            st.markdown("""
                **🟦 Jobs** **🟩 Events** **🟧 Courses**
            """)

            # Google Maps link
            if points["latitude"].notna().any():
                coords = points.head(15)
                maps_query = "|".join([f"{r['latitude']},{r['longitude']}" for _, r in coords.iterrows()])
            else:
                maps_query = "|".join([quote_plus(str(r["district"] or r["location"])) for _, r in results.head(15).iterrows()])
            gmaps_url = f"https://www.google.com/maps/search/?api=1&query={maps_query}"
            st.markdown(f"[📍 Open these locations in Google Maps]({gmaps_url})")
        else:
            st.info("No coordinates available for these results.")
    else:
        st.info("Map will appear once you search something.")

# ------------------------------
# Reset button
# ------------------------------
st.divider()
if st.button("🔄 Clear Chat"):
    st.session_state.results = df.copy()
    st.rerun()


2025-10-22 22:51:40.494 No runtime found, using MemoryCacheStorageManager
2025-10-22 22:51:40.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.546 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.552 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-22 22:51:40.556 Thread 'MainThread':

In [43]:
import streamlit as st
import pandas as pd
import pydeck as pdk
from urllib.parse import quote_plus

# =====================================================
# District centroids (approximate) — instant, no API
# =====================================================
DISTRICT_CENTROIDS = {
    "mitte": (52.5200, 13.4050),
    "kreuzberg": (52.4986, 13.4030),
    "neukölln": (52.4751, 13.4386),
    "friedrichshain": (52.5156, 13.4549),
    "charlottenburg": (52.5070, 13.3040),
    "wilmersdorf": (52.4895, 13.3157),
    "schöneberg": (52.4832, 13.3477),
    "tempelhof": (52.4675, 13.4036),
    "pankow": (52.5693, 13.4010),
    "prenzlauer berg": (52.5380, 13.4247),
    "spandau": (52.5511, 13.1999),
    "steglitz": (52.4560, 13.3220),
    "treptow": (52.4816, 13.4764),
    "köpenick": (52.4429, 13.5756),
    "marzahn": (52.5450, 13.5690),
    "hellersdorf": (52.5345, 13.6132),
    "reinickendorf": (52.5870, 13.3260),
    "moabit": (52.5303, 13.3390),
    "wedding": (52.5496, 13.3551),
    "berlin": (52.5200, 13.4050),   # fallback
}
DISTRICT_KEYS = sorted(DISTRICT_CENTROIDS.keys(), key=len, reverse=True)

# =====================================================
# Helpers: detect district & bake coordinates
# =====================================================
def detect_district(text: str) -> str | None:
    if not isinstance(text, str):
        return None
    lower = text.lower()
    for d in DISTRICT_KEYS:
        if d in lower:
            return d
    if "berlin" in lower:
        return "berlin"
    return None

def to_num(x):
    return pd.to_numeric(x, errors="coerce")

def bake_coords(df_in: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure each row has numeric latitude/longitude instantly:
    - keep existing numeric lat/lon;
    - else use district centroid (from 'district' column or detected in 'location');
    - fallback to Berlin center.
    """
    df = df_in.copy()
    if "latitude" not in df.columns:  df["latitude"] = pd.NA
    if "longitude" not in df.columns: df["longitude"] = pd.NA
    if "district" not in df.columns:  df["district"] = pd.NA

    df["latitude"]  = to_num(df["latitude"])
    df["longitude"] = to_num(df["longitude"])

    needs = df.index[df["latitude"].isna() | df["longitude"].isna()]

    for i in needs:
        row = df.loc[i]
        d = None
        # prefer explicit district column if filled
        if pd.notna(row.get("district")) and str(row["district"]).strip():
            d = detect_district(str(row["district"]))
        # else try the free-text location
        if not d:
            d = detect_district(str(row.get("location", "")))
        if not d:
            d = "berlin"

        lat, lon = DISTRICT_CENTROIDS.get(d, DISTRICT_CENTROIDS["berlin"])
        df.at[i, "latitude"] = lat
        df.at[i, "longitude"] = lon
        # also store cleaned district for display if missing
        if not pd.notna(row.get("district")) or not str(row.get("district")).strip():
            df.at[i, "district"] = d.title()

    return df

# =====================================================
# Load data (and pre-bake coords once)
# =====================================================
@st.cache_data
def load_data():
    base = "backend/data/"
    jobs    = pd.read_csv(base + "berlin_tech_jobs.csv")
    events  = pd.read_csv(base + "berlin_tech_events.csv")
    courses = pd.read_csv(base + "german_courses_berlin.csv")

    for frame, t in [(jobs, "job"), (events, "event"), (courses, "course")]:
        frame.columns = [c.strip().lower() for c in frame.columns]
        frame["type"] = t

    all_df = pd.concat([jobs, events, courses], ignore_index=True)
    all_df = bake_coords(all_df)  # instant coordinates everywhere
    return all_df

df = load_data()

# =====================================================
# App shell
# =====================================================
st.set_page_config(page_title="Kiez Connect", page_icon="💬", layout="wide")
st.title("💬 Kiez Connect – Your Berlin Chat Assistant")
st.caption("Ask about **jobs**, **events**, or **German courses** across Berlin. Instant list + instant pins (no geocoder).")

if "results" not in st.session_state:
    st.session_state.results = df.copy()

col_list, col_map = st.columns([0.45, 0.55])

# =====================================================
# Chat / Query
# =====================================================
query = st.chat_input("Try: 'jobs in Mitte', 'events in Kreuzberg', 'german courses in Charlottenburg'")

if query:
    st.chat_message("user").write(query)
    q = query.lower()

    # topic
    topic = None
    if "job" in q or "work" in q:
        topic = "job"
    elif "event" in q or "meetup" in q or "conference" in q:
        topic = "event"
    elif "course" in q or "german" in q or "class" in q:
        topic = "course"

    # location
    loc_key = detect_district(q)

    # keyword
    keywords = ["developer", "engineer", "data", "design", "marketing", "teacher", "python", "manager"]
    keyword = next((k for k in keywords if k in q), None)

    # filter
    subset = df.copy()
    if topic:
        subset = subset[subset["type"] == topic]
    if loc_key:
        subset = subset[
            subset["district"].fillna("").str.lower().str.contains(loc_key)
            | subset["location"].fillna("").str.lower().str.contains(loc_key)
            | subset["location"].fillna("").str.lower().str.contains("berlin")
        ]
    else:
        subset = subset[
            subset["location"].fillna("").str.lower().str.contains("berlin")
            | subset["district"].notna()
        ]
    if keyword:
        key = keyword.lower()
        search_cols = [c for c in ["title", "company", "provider", "course_name"] if c in subset.columns]
        if search_cols:
            subset = subset[
                subset[search_cols].apply(lambda x: x.astype(str).str.lower().str.contains(key).any(), axis=1)
            ]

    # coords already baked, but enforce again on slice (cheap)
    subset = bake_coords(subset)
    st.session_state.results = subset

    # assistant reply immediately
    if subset.empty:
        st.chat_message("assistant").write("❌ No matches. Try another Berlin district or keyword.")
    else:
        emoji = {"job": "💼", "event": "🎉", "course": "🎓"}.get(topic, "📍")
        area = (loc_key or "berlin").title()
        st.chat_message("assistant").write(
            f"{emoji} Found **{len(subset)} {topic+'s' if topic else 'results'}** around **{area}**. See list + map below."
        )

# =====================================================
# LEFT: Results list (clickable)
# =====================================================
with col_list:
    results = st.session_state.results
    st.subheader(f"Results ({len(results)})" if not results.empty else "Results")

    if results.empty:
        st.info("No results yet. Ask about jobs, events, or German courses in Berlin.")
    else:
        for _, row in results.head(30).iterrows():
            title   = row.get("title", "No title")
            district= row.get("district") or ""
            loc_txt = district if district else row.get("location", "Berlin")
            t       = row.get("type", "")

            # link by type
            link = ""
            if t == "job":
                link = row.get("job_url_direct") or row.get("job_url")
            elif t == "event":
                link = row.get("link")
            elif t == "course":
                link = row.get("url") or row.get("booking_url")

            if isinstance(link, str) and link.startswith("http"):
                st.markdown(f"**[{title}]({link})**  \n{loc_txt}")
            else:
                st.markdown(f"**{title}**  \n{loc_txt}")
            st.markdown("---")

# =====================================================
# RIGHT: PyDeck Map (instant pins, address in tooltip)
# =====================================================
with col_map:
    st.subheader("📍 Map View")

    points = st.session_state.results.dropna(subset=["latitude", "longitude"]).copy()

    if points.empty:
        st.info("No mappable results yet.")
    else:
        # color by type
        def color_for_type(t):
            t = (t or "").lower()
            return {
                "job": [0, 122, 255],    # blue
                "event": [0, 170, 90],   # green
                "course": [255, 140, 0], # orange
            }.get(t, [120, 120, 120])

        points["color"] = points["type"].apply(color_for_type)

        # center map on results
        mid_lat = float(points["latitude"].mean())
        mid_lon = float(points["longitude"].mean())

        # ScatterplotLayer with tooltip that shows title + district + location
        layer = pdk.Layer(
            "ScatterplotLayer",
            data=points,
            get_position="[longitude, latitude]",
            get_fill_color="color",
            get_radius=80,
            pickable=True,
            opacity=0.9,
        )

        tooltip = {
            # shows “address” info (district + location) on hover (fastest UX in Streamlit)
            "html": "<b>{title}</b><br/><i>{district}</i><br/>{location}",
            "style": {"backgroundColor": "white", "color": "black"},
        }

        view_state = pdk.ViewState(latitude=mid_lat, longitude=mid_lon, zoom=11)
        deck = pdk.Deck(
            layers=[layer],
            initial_view_state=view_state,
            tooltip=tooltip,
            map_style="light"
        )
        st.pydeck_chart(deck, use_container_width=True)

        st.markdown("**🟦 Jobs** **🟩 Events** **🟧 Courses**")

        # “Open in Google Maps” for up to 20 pins
        top_points = points.head(20)
        maps_query = "|".join([f"{r['latitude']},{r['longitude']}" for _, r in top_points.iterrows()])
        gmaps_url = f"https://www.google.com/maps/search/?api=1&query={maps_query}"
        st.markdown(f"[📍 Open these locations in Google Maps]({gmaps_url})")

# =====================================================
# Reset
# =====================================================
st.divider()
if st.button("🔄 Clear Chat"):
    st.session_state.results = df.copy()
    st.rerun()


2025-10-23 05:55:36.785 No runtime found, using MemoryCacheStorageManager
2025-10-23 05:55:36.797 No runtime found, using MemoryCacheStorageManager
2025-10-23 05:55:36.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 05:55:36.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 05:55:36.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 05:55:36.806 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 05:55:36.914 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 05:55:36.918 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 05:55:36.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

In [ ]:
python -m streamlit run app.py

In [34]:
!pip install pydeck

Defaulting to user installation because normal site-packages is not writeable


In [ ]:

#!pip install geopy

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import os
from pathlib import Path
import pandas as pd
import streamlit as st
import pydeck as pdk

# =====================================================
# Robust data directory resolver (works everywhere)
# =====================================================
def resolve_data_dir() -> Path:
    """
    Find backend/data folder safely.
    Works in Streamlit, Python, and Jupyter (no __file__ issues).
    """
    # 1) Environment variable override
    env = os.environ.get("KC_DATA_DIR")
    if env and Path(env).exists():
        return Path(env)

    # 2) Base dir depends on whether __file__ exists
    try:
        base = Path(__file__).resolve().parent
    except NameError:
        base = Path(os.getcwd())

    # 3) Check possible locations
    candidates = [
        base / "backend" / "data",
        base / "data",
        Path.cwd() / "backend" / "data",
        Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect\backend\data"),
    ]
    for c in candidates:
        if c.exists():
            return c

    return base / "backend" / "data"

DATA_DIR = resolve_data_dir()

# =====================================================
# Berlin District Coordinates (preloaded fallback)
# =====================================================
DISTRICT_CENTROIDS = {
    "mitte": (52.5200, 13.4050),
    "kreuzberg": (52.4986, 13.4030),
    "neukölln": (52.4751, 13.4386),
    "friedrichshain": (52.5156, 13.4549),
    "charlottenburg": (52.5070, 13.3040),
    "wilmersdorf": (52.4895, 13.3157),
    "schöneberg": (52.4832, 13.3477),
    "tempelhof": (52.4675, 13.4036),
    "pankow": (52.5693, 13.4010),
    "prenzlauer berg": (52.5380, 13.4247),
    "spandau": (52.5511, 13.1999),
    "steglitz": (52.4560, 13.3220),
    "treptow": (52.4816, 13.4764),
    "köpenick": (52.4429, 13.5756),
    "marzahn": (52.5450, 13.5690),
    "hellersdorf": (52.5345, 13.6132),
    "reinickendorf": (52.5870, 13.3260),
    "moabit": (52.5303, 13.3390),
    "wedding": (52.5496, 13.3551),
    "berlin": (52.5200, 13.4050),
}
DISTRICT_KEYS = sorted(DISTRICT_CENTROIDS.keys(), key=len, reverse=True)

# =====================================================
# Helpers
# =====================================================
def detect_district(text: str):
    if not isinstance(text, str):
        return None
    lower = text.lower()
    for d in DISTRICT_KEYS:
        if d in lower:
            return d
    if "berlin" in lower:
        return "berlin"
    return None


def bake_coords(df_in: pd.DataFrame) -> pd.DataFrame:
    """Ensure lat/lon exist, using district approximations for missing ones."""
    df = df_in.copy()
    if "latitude" not in df.columns:
        df["latitude"] = pd.NA
    if "longitude" not in df.columns:
        df["longitude"] = pd.NA
    if "district" not in df.columns:
        df["district"] = pd.NA

    for i in df.index:
        if pd.isna(df.at[i, "latitude"]) or pd.isna(df.at[i, "longitude"]):
            d = (
                detect_district(str(df.at[i, "district"]))
                or detect_district(str(df.at[i, "location"]))
                or "berlin"
            )
            lat, lon = DISTRICT_CENTROIDS.get(d, DISTRICT_CENTROIDS["berlin"])
            df.at[i, "latitude"] = lat
            df.at[i, "longitude"] = lon
            if pd.isna(df.at[i, "district"]) or not str(df.at[i, "district"]).strip():
                df.at[i, "district"] = d.title()
    return df


# =====================================================
# Load data safely
# =====================================================
@st.cache_data
def load_data(data_dir: Path):
    try:
        jobs = pd.read_csv(data_dir / "berlin_tech_jobs.csv")
        events = pd.read_csv(data_dir / "berlin_tech_events.csv")
        courses = pd.read_csv(data_dir / "german_courses_berlin.csv")
    except Exception as e:
        st.error(f"❌ Could not load CSV files from {data_dir}\n\nError: {e}")
        return pd.DataFrame()

    for df, t in [(jobs, "job"), (events, "event"), (courses, "course")]:
        df.columns = [c.strip().lower() for c in df.columns]
        df["type"] = t

    merged = pd.concat([jobs, events, courses], ignore_index=True)
    merged = bake_coords(merged)
    return merged


# =====================================================
# Streamlit UI
# =====================================================
st.set_page_config(page_title="Kiez Connect", page_icon="💬", layout="wide")
st.title("💬 Kiez Connect – Your Berlin Chat Assistant")
st.caption("Ask about jobs, events, or German courses — instant results and map pins across Berlin.")

df = load_data(DATA_DIR)

if df.empty:
    st.warning("⚠️ No data loaded. Check your CSV files in backend/data.")
    st.stop()

if "results" not in st.session_state:
    st.session_state.results = df.copy()

col_list, col_map = st.columns([0.45, 0.55])

# =====================================================
# Chat Input
# =====================================================
query = st.chat_input("Example: 'show jobs in Mitte' or 'find events in Kreuzberg'")

if query:
    st.chat_message("user").write(query)
    q = query.lower()

    topic = None
    if "job" in q:
        topic = "job"
    elif "event" in q:
        topic = "event"
    elif "course" in q or "german" in q:
        topic = "course"

    loc_key = detect_district(q)
    keywords = ["developer", "engineer", "data", "design", "marketing", "teacher", "python", "manager"]
    keyword = next((k for k in keywords if k in q), None)

    subset = df.copy()
    if topic:
        subset = subset[subset["type"] == topic]
    if loc_key:
        subset = subset[
            subset["district"].fillna("").str.lower().str.contains(loc_key)
            | subset["location"].fillna("").str.lower().str.contains(loc_key)
        ]
    if keyword:
        search_cols = [c for c in ["title", "company", "provider", "course_name"] if c in subset.columns]
        subset = subset[
            subset[search_cols].apply(lambda x: x.astype(str).str.lower().str.contains(keyword).any(), axis=1)
        ]

    subset = bake_coords(subset)
    st.session_state.results = subset

    if subset.empty:
        st.chat_message("assistant").write("❌ No matches found. Try another keyword or district.")
    else:
        emoji = {"job": "💼", "event": "🎉", "course": "🎓"}.get(topic, "📍")
        st.chat_message("assistant").write(
            f"{emoji} Found **{len(subset)} {topic}s** in **{(loc_key or 'Berlin').title()}**."
        )


# =====================================================
# Left Column – Results List
# =====================================================
with col_list:
    results = st.session_state.results
    st.subheader(f"Results ({len(results)})")

    if results.empty:
        st.info("No results yet.")
    else:
        for _, row in results.head(25).iterrows():
            title = row.get("title", "No title")
            location = row.get("district") or row.get("location") or "Berlin"
            link = (
                row.get("job_url_direct")
                or row.get("job_url")
                or row.get("link")
                or row.get("url")
            )
            if isinstance(link, str) and link.startswith("http"):
                st.markdown(f"**[{title}]({link})**  \n📍 {location}")
            else:
                st.markdown(f"**{title}**  \n📍 {location}")
            st.markdown("---")


# =====================================================
# Right Column – Map View
# =====================================================
with col_map:
    st.subheader("📍 Map View")

    # ✅ Defensive fix: make sure coordinates exist before filtering
    results = st.session_state.results.copy()

    # Create latitude/longitude columns if missing
    if "latitude" not in results.columns or "longitude" not in results.columns:
        results = bake_coords(results)

    # Fill coordinates if they’re still empty
    results["latitude"] = results["latitude"].fillna(52.5200)
    results["longitude"] = results["longitude"].fillna(13.4050)

    # Now drop rows without valid coordinates
    points = results.dropna(subset=["latitude", "longitude"])

    if points.empty:
        st.info("No map points available yet.")
    else:
        def color_for_type(t):
            t = (t or "").lower()
            return {
                "job": [0, 122, 255],
                "event": [0, 170, 90],
                "course": [255, 140, 0],
            }.get(t, [128, 128, 128])

        points["color"] = points["type"].apply(color_for_type)
        mid_lat = float(points["latitude"].mean())
        mid_lon = float(points["longitude"].mean())

        layer = pdk.Layer(
            "ScatterplotLayer",
            data=points,
            get_position="[longitude, latitude]",
            get_fill_color="color",
            get_radius=80,
            pickable=True,
        )

        tooltip = {"html": "<b>{title}</b><br/>{district}<br/>{location}"}
        view_state = pdk.ViewState(latitude=mid_lat, longitude=mid_lon, zoom=11)
        st.pydeck_chart(pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip=tooltip))

st.divider()
if st.button("🔄 Clear Chat"):
    st.session_state.results = df.copy()
    st.rerun()


2025-10-23 07:34:36.437 No runtime found, using MemoryCacheStorageManager
2025-10-23 07:34:36.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.446 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-23 07:34:36.461 Thread 'MainThread':